----

In [1]:
from datetime import datetime

In [2]:
import requests
import json
import os
from pprint import pprint

In [3]:
def download_url(url):
    headers = {
    "accept":"application/json",
    "content-type":"application/json",
    "x-api-key":"3d996743-0e66-4c37-b4c6-20f309040d87",
    "if-none-match":"0cfeec2f23a3cff3faff5f5d08f860e42",
    "user-agent":"NTC-ios/7.9.0",
    "accept-language":"fr-fr",
    }

    resp = requests.get(url,headers=headers)
    return resp.json()

In [7]:
def get_all_matches_for_team(team_id):
    url = f"https://match.uefa.com/v2/matches?language=FR&offset=0&limit=20&style=FULL&teamId={team_id}"
    all_matches = download_url(url)


    passed_matches = []
    upcoming_matches = []
    for m in all_matches:

        is_passed = datetime.strptime(m['kickOffTime']['date'], '%Y-%m-%d')  < datetime.now()

        new_match = {
            'id':m['id'],
            'away_team':m['awayTeam']['countryCode'],
            'home_team':m['homeTeam']['countryCode'],
            'competition_code':m['competition']['code'],
            'competition_id':m['competition']['id'],
            'competition_name':m['competition']['metaData']['name'],
            'competition_times':m['kickOffTime'],
        }
        if is_passed:
            passed_matches.append(new_match)
        else:
            upcoming_matches.append(new_match)
            
    return (passed_matches, upcoming_matches)

In [42]:

def download_events_for_match(match_id):
    url = f'''https://editorial.uefa.com/api/cachedsearch/build?language=FR
    &aggregator=lightnodejson
    &limit=20
    &offset=0
    &type=liveblog
    &sorting=-attributes.firstPublicationDate
    &param.attributes.language=fr
    &param.attributes.footballEntities.match.matchId={match_id}
    &param%5Battributes%5D%5BhideFromApp%21%5D=true'''.replace("\n","").replace(' ', '')

    #print(url)
    match_data = download_url(url)

    match_editorial_id = match_data['result'][0]['nodeData']['id']
    
    # Next step : rebuild all events
    #events_idxed = {}
    events = []
    last_id = None
    while True:

        if last_id is not None:
            url = f"https://editorial.uefa.com/api/liveblogs/{match_editorial_id}/posts?language=FR&aggregator=lightliveblogjson&limit=40&from={last_id}"
        else:
            url = f"https://editorial.uefa.com/api/liveblogs/{match_editorial_id}/posts?language=FR&aggregator=lightliveblogjson&limit=40"
        #print(url)
        match_data = download_url(url)

        for item in match_data['result']:
            if item['attributes']['liveblogPostData']['lbPostType'] == 'MATCH_EVENT':
                #print(item['id'])
                #print(item['attributes']['liveblogPostData']['lbPostTimestamp'])

                event_type = item['attributes']['liveblogPostData']['lbPostEvent']['eventType']
                event_translated =  item['attributes']['liveblogPostData']['lbPostEvent']['eventTranslation']

                #if event_type not in events_idxed:
                #    events_idxed[event_type] = []
                #events_idxed[event_type].append(event_translated)

                events.insert(0, {'id':item['id'], 
                                  'timestamp':item['attributes']['liveblogPostData']['lbPostTimestamp'],
                                  'event_type':event_type,
                                  'event_translated':event_translated,
                                  'liveblog_raw':item['attributes']['liveblogPostData']
                                 }) 
            last_id = item['id']
        if match_data['count'] < 40:
            print(f" Expected 40 items, go {match_data['count']}. Break")
            break
       
    return events

In [40]:
team_id = 43 # France
#team_id = 66 # Italy
#team_id = 127 # Sweden
(passed_matches, upcoming_matches) = get_all_matches_for_team(team_id)

In [35]:
for i in range(len(passed_matches)):
    print(passed_matches[i]['id'])
    print(passed_matches[i])
    print()

2032159
{'id': '2032159', 'away_team': 'ARM', 'home_team': 'SWE', 'competition_code': 'A Int.', 'competition_id': '19', 'competition_name': 'Friendly Matches', 'competition_times': {'date': '2021-06-05', 'dateTime': '2021-06-05T18:45:00Z', 'utcOffsetInHours': 2}}

2032177
{'id': '2032177', 'away_team': 'FIN', 'home_team': 'SWE', 'competition_code': 'A Int.', 'competition_id': '19', 'competition_name': 'Friendly Matches', 'competition_times': {'date': '2021-05-29', 'dateTime': '2021-05-29T16:00:00Z', 'utcOffsetInHours': 2}}

2031953
{'id': '2031953', 'away_team': 'EST', 'home_team': 'SWE', 'competition_code': 'A Int.', 'competition_id': '19', 'competition_name': 'Friendly Matches', 'competition_times': {'date': '2021-03-31', 'dateTime': '2021-03-31T15:45:00Z', 'utcOffsetInHours': 2}}

2030810
{'id': '2030810', 'away_team': 'SWE', 'home_team': 'KOS', 'competition_code': 'WC', 'competition_id': '17', 'competition_name': 'FIFA World Cup', 'competition_times': {'date': '2021-03-28', 'dateTi

In [41]:
events_translation = { "1":"red_card",
                        "2":"red_card",
                        #"3":"goal", # may contain "penalty"
                        "4":"player_switch", #changement
                        "5":"missed_penalty", 
                        "6":"stopped_shot",
                        "7":"out_of_frame_shot",
                        "8":"framed_shot",
                        "9":"offside", 
                        "10":"corner_shot",
                        "11":"foul",  # may contain penalty 
                        "12":"assist", # passe décisive
                        "13":"phase_start", 
                        "14":"phase_end",
                        "19":"additional_time",
                        "30":"free_kick",
                        "32":"stop", # arrêt par le gardien
                        "80":"corner_concealed"
                     }

os.makedirs(f"data/matches/", exist_ok=True)

last_matches_ids = [m['id'] for m in passed_matches]

for match_id in last_matches_ids: 
    print(match_id)
    
    fpath = f"data/matches/{match_id}_events.json"
    if os.path.isfile(fpath) and False:
        print("loading cached data")
        with open(fpath, "r") as f:
            events = json.loads(f.read())
            
    else:
        print("downloading data")        
        events = download_events_for_match(match_id)
        with open(fpath, "w") as f:
            f.write(json.dumps(events))


    for e in events:

        #if e['event_type'] in events_translation:
        #    continue

        if 'penalty' not in e['event_translated']:
            continue
        
        print(e['event_type'], e['event_translated'])

        print("Mi-Temps #%s"%(e['liveblog_raw']['lbPostEvent']['eventPhase'])  , 
              "%s:%s"%(e['liveblog_raw']['lbPostEvent']['eventMinute'], e['liveblog_raw']['lbPostEvent']['eventSecond'])      )
        try:
            print(e['liveblog_raw']['lbPostEvent']['idPlayer'], " -> ", e['liveblog_raw']['lbPostEvent']['idPlayerTo'] )
            print(e['liveblog_raw']['lbPostEvent']['idTeam'], " -> ",   e['liveblog_raw']['lbPostEvent']['idTeamTo']  )
            print(e['liveblog_raw']['lbPostEvent']['namePlayer'] , " -> ",  e['liveblog_raw']['lbPostEvent']['namePlayerTo']  )
        except:
            try:
                print(e['liveblog_raw']['lbPostEvent']['idPlayer']  )
                print(e['liveblog_raw']['lbPostEvent']['idTeam']  )
                print(e['liveblog_raw']['lbPostEvent']['namePlayer']  )
            except:
                pass

        #if e['event_type'] in ("19"):
        #    pprint(e)
    print("----")

2032240
downloading data
https://editorial.uefa.com/api/cachedsearch/build?language=FR&aggregator=lightnodejson&limit=20&offset=0&type=liveblog&sorting=-attributes.firstPublicationDate&param.attributes.language=fr&param.attributes.footballEntities.match.matchId=2032240&param%5Battributes%5D%5BhideFromApp%21%5D=true


TypeError: 'NoneType' object is not iterable

### ----
## Below is raw research

Click sur France => 



# This url allows to build the second one.
# it also gives extra perks like the referees and stuff


https://editorial.uefa.com/api/cachedsearch/build?language=FR&aggregator=lightnodejson&limit=10
&offset=0
&type=videostory
&sorting=-attributes.firstPublicationDate
&param.attributes.language=fr
&param.attributes.main.kind=Highlights
&param.attributes.main.kind=Medium+Highlights
&param.attributes.footballEntities.match.matchId=2032240
&param.attributes.footballEntities.match.matchId=2030818
&param.attributes.footballEntities.match.matchId=2030791
&param.attributes.footballEntities.match.matchId=2030762
&param.attributes.footballEntities.match.matchId=2030108
&param.attributes.footballEntities.match.matchId=2030080
&param.attributes.footballEntities.match.matchId=2029167
&param.attributes.footballEntities.match.matchId=2030063
&param.attributes.footballEntities.match.matchId=2030032
&param.attributes.footballEntities.match.matchId=2029150
&param%5Battributes%5D%5BhideFromApp%21%5D=true
# this url doesn't give the detailled flow of an event


https://editorial.uefa.com/api/cachedsearch/build?language=FR&aggregator=lightnodejson&limit=20
&offset=0
&type=article
&type=videostory
&sorting=-attributes.firstPublicationDate
&param.attributes.language=fr
&param.attributes.footballEntities.team.teamId=43
&param%5Battributes%5D%5BhideFromApp%21%5D=true

In [82]:
lineup = download_url("https://match.uefa.com/v3/matches/2032240/lineups?language=FR")

test = download_url("https://match.uefa.com/v2/matches/2032240/events?language=FR&filter=LINEUP&offset=0&limit=50")

In [85]:
pprint(test)

[{'fieldPosition': {},
  'id': 'b88eb0d0b0914a33856c15842db71385',
  'matchId': '2032240',
  'phase': 'SECOND_HALF',
  'primaryActor': {'person': {'age': '28',
                              'birthDate': '1992-08-08',
                              'detailedFieldPosition': 'UNKNOWN',
                              'fieldPosition': 'FORWARD',
                              'id': '250130512',
                              'imageUrl': 'https://img.uefa.com/imgml/TP/players/19/2022/324x324/250130512.jpg',
                              'internationalName': 'Kieffer Moore',
                              'translations': {'fieldPosition': {'AZ': 'Forward',
                                                                 'DA': 'Forward',
                                                                 'DE': 'Stürmer',
                                                                 'EN': 'Forward',
                                                                 'ES': 'Delantero',
                 

In [83]:
#print(lineup['lineupStatus'])
#print(lineup['matchId'])

print(lineup['awayTeam']['shirtColor'])
print(lineup['homeTeam']['shirtColor'])


# for p in lineup['awayTeam']['field']:
for p in lineup['homeTeam']['field']:
    print(p['fieldCoordinate'])
    print(p['type'])  # can be : PLAYER, CAPTAIN, GOALKEEPER, GOALKEEPER_CAPTAIN
    print(p['player']['internationalName'], f"#{p['jerseyNumber']}", "\t",  p['player']['id'], )
    print()

'''
for p in lineup['homeTeam']['bench']:
    #print(p['type'])  # can be : PLAYER, CAPTAIN, GOALKEEPER, GOALKEEPER_CAPTAIN
    #print(p['player']['internationalName'], f"#{p['jerseyNumber']}", "\t",  p['player']['id'], )
    pprint(p)
'''
    

#FF0000
#0000A0
{'x': 500, 'y': 50}
GOALKEEPER_CAPTAIN
Hugo Lloris #1 	 93846

{'x': 200, 'y': 250}
PLAYER
Benjamin Pavard #2 	 250081921

{'x': 600, 'y': 250}
PLAYER
Presnel Kimpembe #3 	 250064583

{'x': 400, 'y': 250}
PLAYER
Raphaël Varane #4 	 250025891

{'x': 300, 'y': 500}
PLAYER
Paul Pogba #6 	 250015808

{'x': 300, 'y': 800}
PLAYER
Antoine Griezmann #7 	 250019498

{'x': 700, 'y': 800}
PLAYER
Kylian Mbappé #10 	 250076574

{'x': 500, 'y': 500}
PLAYER
Corentin Tolisso #12 	 250054829

{'x': 700, 'y': 500}
PLAYER
Adrien Rabiot #14 	 250053905

{'x': 500, 'y': 800}
PLAYER
Karim Benzema #19 	 93321

{'x': 800, 'y': 250}
PLAYER
Lucas Hernández #21 	 250063803



'\nfor p in lineup[\'homeTeam\'][\'bench\']:\n    #print(p[\'type\'])  # can be : PLAYER, CAPTAIN, GOALKEEPER, GOALKEEPER_CAPTAIN\n    #print(p[\'player\'][\'internationalName\'], f"#{p[\'jerseyNumber\']}", "\t",  p[\'player\'][\'id\'], )\n    pprint(p)\n'